## Using Komo for IK

In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/jung-su/git/robotics-course/course3-Simulation/03-motion'** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '


**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}
** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/jung-su/git/robotics-course/course3-Simulation/03-motion'


** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

In [3]:
obj = C.addFrame("object")
obj.setPosition([1., 0., 1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

In [4]:
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.2]);

In [5]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220


{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 0.7709668210737692},
 'F_qQuaternionNorms/0-#110': {'order': 0.0, 'type': 'eq'},
 'F_PositionRel/0-object-R_gripperCenter': {'order': 0.0,
  'type': 'eq',
  'eq': 0.00031254232040027474},
 'sos': 0.778676489284507,
 'ineq': 0.0,
 'eq': 0.00031254232040027474,
 'f': 0.0}

** optimization time:0.005198 (kin:0.000195 coll:0 feat:0.001569 newton: 0.001866) setJointStateCount:10
   sos:0.778676 ineq:0 eq:0.000312542


In [6]:
C.setFrameState( IK.getFrameState(0) )

In [10]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# # copy C into the IK's internal configuration(s)
# IK.setConfigurations(C)

# redefine the IK problem
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.1]);
# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )

{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 0.06740781678702858}, 'F_qQuaternionNorms/0-#110': {'order': 0.0, 'type': 'eq'}, 'F_PositionRel/0-object-R_gripperCenter': {'order': 0.0, 'type': 'eq', 'eq': 0.0021237462531570675}, 'sos': 0.06808189495489886, 'ineq': 0.0, 'eq': 0.0021237462531570675, 'f': 0.0}
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000435 (kin:2e-05 coll:0 feat:0.000155 newton: 3.2e-05) setJointStateCount:4
   sos:0.0680819 ineq:0 eq:0.00212375


## Path Optimization

In [11]:
obj.setPosition([.8,.2,1.5])

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,40, 5., True)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2])

komo.optimize()
komo.getReport()

{'F_qItself/2-#28': {'order': 2.0, 'type': 'sos', 'sos': 0.08934823365829975},
 'F_PositionDiff/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'sos',
  'sos': 6.134460646622827e-05},
 'F_qItself/1-#28': {'order': 1.0, 'type': 'eq', 'eq': 1.5060678071679434e-05},
 'F_PairCollision/0-R_gripper-object': {'order': 0.0, 'type': 'ineq'},
 'sos': 0.08940957826476598,
 'ineq': 0.0,
 'eq': 1.5060678071679434e-05,
 'f': 0.0}

** KOMO::run solver:sparse collisions:1 x-dim:560 T:40 k:2 phases:1 stepsPerPhase:40 tau:0.125  #timeSlices:42 #totalDOFs:560 #frames:4620
** optimization time:0.149016 (kin:0.008609 coll:0.110415 feat:0.015129 newton: 0.007431) setJointStateCount:15
   sos:0.0894096 ineq:0 eq:1.50607e-05


In [12]:
V = komo.view_play(True, 0.1)

In [13]:
C.setFrameState(komo.getFrameState(19))
C.getJointState()
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)

(array([0.]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))